In [11]:
#import requiered libraries
import os
import warnings
import sys

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse


In [12]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from train_classifiers import TrainingClassifier 

In [17]:
cleaned_df  = pd.read_csv('../data/clean_data.csv')

In [18]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1243 non-null   int64 
 1   auction_id   1243 non-null   object
 2   experiment   1243 non-null   object
 3   date         1243 non-null   object
 4   hour         1243 non-null   int64 
 5   device_make  1243 non-null   object
 6   platform_os  1243 non-null   int64 
 7   browser      1243 non-null   object
 8   aware        1243 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 87.5+ KB


In [21]:
def get_data(tag, path='data/AdSmartABdata.csv.csv', repo='https://github.com/Group-12-A-B-Hypothesis-Testing/abtest-mlops'):
    rev = tag
    data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
    df = pd.read_csv(data_url)
    return df

In [26]:
path = 'data/AdSmartABdata.csv'
repo = 'https://github.com/Group-12-A-B-Hypothesis-Testing/abtest-mlops'
version ='e19300fae64a8d41f40122d8f68a53'

In [27]:
data_url = dvc.api.get_url(
    path=path,
    repo=repo,
    rev=version
)

RevError: unknown Git revision 'e19300fae64a8d41f40122d8f68a53'